In [1]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import cv2

import requests
from io import BytesIO
from PIL import Image
import numpy as np

from moviepy.editor import VideoFileClip

# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.structures.keypoint import keypoints_to_heat_map
from maskrcnn_benchmark.modeling.roi_heads.keypoint_head.loss import project_keypoints_to_heatmap
from predictor import COCODemo

In [2]:
config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'bee', \
                     'OUTPUT_DIR', 'training_dir_v2/fcos_kps_ms_training_R_50_FPN_1x_bee1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir_v2/fcos_kps_ms_training_R_50_FPN_1x_bee1/model_final.pth',\
                     'DATASETS.TEST', "('bee_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                    ])



In [3]:
coco_demo = COCODemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.2,
)


In [4]:
def load(img_path):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    pil_image = Image.open(img_path).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_video(video_path,frame_id=[0]):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    video_clip = VideoFileClip(video_path)
    if frame_id is None:
        nframes = int(video_clip.fps * video_clip.duration)
        frame_id = range(nframes)
    images = []
    for i in frame_id:
        image = video_clip.get_frame(i/video_clip.fps)
        # convert to BGR format
        image = np.array(image)[:, :, [2, 1, 0]]
        images += [image]
    video_clip.close()
    
    return images

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [5]:
#keypoints_to_heat_map(1,1,1)

In [6]:
#test_file = 'bee_test.mp4'
#frame_index = None
#images = load_video(test_file,frame_index)

#save_file = 'bee_test_pred.mp4'
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#dd1,dd2 = images[0].shape[0],images[0].shape[1]
#out = cv2.VideoWriter(save_file, fourcc, 20.0, (dd2,dd1))
#for image in images:
    # compute predictions
 #   predictions = coco_demo.run_on_opencv_image(image)

  #  out.write(predictions[:, :, [2, 1, 0]].astype(np.uint8))
#out.release()



In [7]:
#BUILDING OWN VISUALIZER

#1. obtain heatmap
#2. use function to get peaks
#3. Overlay peaks onto image
#4. voila

In [8]:
#test_file = 'bee_test.mp4'
#frame_index = None
#images = load_video(test_file,frame_index)

#save_file = 'bee_test_pred.mp4'
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#dd1,dd2 = images[0].shape[0],images[0].shape[1]
#out = cv2.VideoWriter(save_file, fourcc, 20.0, (dd2,dd1))
# compute predictions
#predictions, tp, hm_res, stride, hms = coco_demo.run_on_opencv_image(images[0])
#print(predictions)

#    out.write(predictions[:, :, [2, 1, 0]].astype(np.uint8))
#out.release()


In [9]:
#print(hms)

In [10]:
#print(tp.get_field('keypoints'))

In [11]:
#kps = tp.get_field('keypoints')
#ds = cfg.MODEL.ROI_MASK_HEAD.RESOLUTION

In [12]:
#(keypoints, rois, heatmap_size)
#keypoints_to_heat_map(kps,,1)

In [13]:
#hms, valid = project_keypoints_to_heatmap(kps, tp, ds)

In [14]:
# num keypoints per animal, heatmap
def calculate_peaks(numparts, heatmap_avg):
    score = np.ones((numparts,)) * 0.000001
    all_peaks = []
    peak_counter = 0
    if len(score) < numparts:
        score = score[:numparts]
        ##logger.ERROR(‘Not enough scores provided for number of parts’)
        # return
    # threshold_detection = params[‘thre1’]
    # tic_localmax=time.time()
    for part in range(numparts):
        map_ori = heatmap_avg[:, :, part]
        map = map_ori
        map_left = np.zeros(map.shape)
        map_left[1:, :] = map[:-1, :]
        map_right = np.zeros(map.shape)
        map_right[:-1, :] = map[1:, :]
        map_up = np.zeros(map.shape)
        map_up[:, 1:] = map[:, :-1]
        map_down = np.zeros(map.shape)
        map_down[:, :-1] = map[:, 1:]
        peaks_binary = np.logical_and(np.logical_and(np.logical_and(map >= map_left, map >= map_right),
                                                     np.logical_and(map >= map_up, map >= map_down)), map > score[part])
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0]))  # note reverse
        peaks_with_score_and_id = [x + (map_ori[x[1], x[0]], i + peak_counter,) for i, x in
                                   enumerate(peaks)]  # if x[0]>0 and x[1]>0 ]
        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    return all_peaks



In [15]:
test_file = 'bee_test.mp4'
frame_index = None
images = load_video(test_file,frame_index)
image = images[10]
ogimg = images[10]
# imshow(image)
# compute predictions
predictions, tps, hm_res, stride, hms, targets = coco_demo.run_on_opencv_image(image)
np.shape(hms)
hms_tensor = hms.cpu().data.numpy()   
peaks = calculate_peaks(5, hms_tensor)    
    


coords
tensor([[  0,   0,   3,  62],
        [  0,   0,   3, 117],
        [  0,   0,  30,  42],
        [  0,   0,  40,  58],
        [  0,   0,  42,  58],
        [  0,   0,  47,  47],
        [  0,   0,  59,  62],
        [  0,   0,  65, 105],
        [  0,   0,  71,  34],
        [  0,   0,  75, 129],
        [  0,   0,  78,  69],
        [  0,   0,  84, 142],
        [  0,   1,  40,  57],
        [  0,   1,  42,  32],
        [  0,   1,  50,  71],
        [  0,   1,  55,  38],
        [  0,   1,  60,  45],
        [  0,   1,  60,  69],
        [  0,   1,  70, 149],
        [  0,   1,  79,  92],
        [  0,   1,  89, 119],
        [  0,   2,  37,  52],
        [  0,   2,  44,  36],
        [  0,   2,  49,  67],
        [  0,   2,  60,  34],
        [  0,   2,  60,  49],
        [  0,   2,  65,  69],
        [  0,   2,  71, 145],
        [  0,   2,  75, 144],
        [  0,   2,  76,  96],
        [  0,   2,  84, 124],
        [  0,   3,  37,  32],
        [  0,   3,  42,  58],
   

In [ ]:
print(np.shape(predictions))
print(np.shape(targets))

In [ ]:
filename = 'bee_dp_labeled.jpg'
cv2.imwrite(filename, predictions[:, :, [2, 1, 0]]) 

In [ ]:
print(np.shape(hm_res))
print(np.shape(hms_tensor))
print(np.ndim(hm_res))
print(np.ndim(hms_tensor))
print((hm_res[0][0]))
print(type(hm_res[0][0]))

In [ ]:
print(hms_tensor[0][0])
print(np.shape(hms_tensor[0][0]))
print(np.ndim(hms_tensor[0][0]))

In [ ]:
print(np.shape(hms_tensor))
print(np.shape(hms_tensor[0][1]))
#print(np.ndim(hms_tensor[0][0][0][0].cpu().data.numpy()))

In [ ]:
#print(hm_res)

In [ ]:
from maskrcnn_benchmark.utils import cv2_util
import matplotlib.pyplot as plt
#cv2.applyColorMap(np.float32(hms_tensor[0][0]), cv2.COLORMAP_JET)

In [ ]:
import numpy as np
import math

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

# test
print(sigmoid_v(hms_tensor[0][0]))

In [ ]:
# First keypoint (tail)
#sigmoid_v
#arr1 = (hms_tensor[0][0]).astype(np.uint8)
#arr1 = cv2.applyColorMap(arr1, cv2.COLORMAP_MAGMA)
#imgplotog1 = plt.imshow(ogimg)
#plt.show()
#imgplot1 = plt.imshow(arr1)
#plt.show()


In [ ]:
#tail keypoint
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
fig, ax = plt.subplots()
data = gaussian_filter(sigmoid_v(hms_tensor[0][0]), sigma=1)
ax = plt.gca()
ax.invert_yaxis()
plt.pcolormesh(data, cmap='YlOrRd', shading='gouraud')
fig.canvas.draw()
plt.pause(5)
#arr1 = (data).astype(np.uint8)
#arr1 = cv2.applyColorMap(arr1, cv2.COLORMAP_MAGMA)
imgplotog1 = plt.imshow(ogimg)
#plt.show()
#imgplot1 = plt.imshow(arr1)
plt.show()
#print(data)

In [ ]:
# Second keypoint (head)
arr2 = hms_tensor[0][1].astype(np.uint8)
arr2 = cv2.applyColorMap(arr2, cv2.COLORMAP_JET)
imgplotog2 = plt.imshow(ogimg)
plt.show()
imgplot2 = plt.imshow(arr2)
plt.show()

In [ ]:
# Third keypoint (thorax)
arr3 = hms_tensor[0][2].astype(np.uint8)
arr1 = cv2.applyColorMap(arr8, cv2.COLORMAP_JET)
imgplotog1 = plt.imshow(ogimg)
plt.show()
imgplot1 = plt.imshow(arr1)
plt.show()

In [ ]:
# First keypoint (right antena)
arr1 = hms_tensor[0][0].astype(np.uint8)
arr1 = cv2.applyColorMap(arr8, cv2.COLORMAP_JET)
imgplotog1 = plt.imshow(ogimg)
plt.show()
imgplot1 = plt.imshow(arr1)
plt.show()

In [ ]:
# Fifth keypoint (left antena)
arr1 = hms_tensor[0][0].astype(np.uint8)
arr1 = cv2.applyColorMap(arr8, cv2.COLORMAP_JET)
imgplotog1 = plt.imshow(ogimg)
plt.show()
imgplot1 = plt.imshow(arr1)
plt.show()

In [ ]:
filename = 'AAA_B1'+'.jpg'
cv2.imwrite(filename, predictions[:, :, [2, 1, 0]]) 

In [ ]:
np.shape(peaks[4])

In [ ]:
import torch
masks_per_dim = 5
result = torch.zeros(
            (masks_per_dim * 800, masks_per_dim * 1422), dtype=torch.uint8)

In [ ]:
np.shape(result.numpy())

In [ ]:
print(stride)